In [1]:
!pip install torch_geometric

In [2]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-sparse 0.6.0
    Uninstalling torch-sparse-0.6.0:
      Successfully uninstalled torch-sparse-0.6.0


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import time
import torch.nn.functional as F

In [0]:
from utils import EarlyStopping

In [0]:
from torch_geometric.datasets import Planetoid

In [0]:
Cora = Planetoid(root='/content/drive/My Drive/', name='Cora')
cora = Cora[0]

In [0]:
C_S = Planetoid(root='/content/drive/My Drive/', name='CiteSeer')
cite_seer = C_S[0]

In [0]:
PM = Planetoid(root='/content/drive/My Drive/', name='PubMed')
pub_med = PM[0]

In [0]:
# for compress test
cora = torch.load('/content/drive/My Drive/cora_193.pt')

In [26]:
cora

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 193], y=[2708])

In [0]:
data = cora
#data.x = cora.x[:, :800] for compress test
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
from torch_geometric.nn import GCNConv, AGNNConv, SAGEConv,GATConv,GINConv, ChebConv

In [28]:
num_features

193

## GCN

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 64, cached=True)
        self.conv2 = GCNConv(64, num_classes, cached=True)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)

        #self.reg_params = self.conv1.parameters()
        #self.non_reg_params = self.conv2.parameters()
    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

In [40]:
gcns = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  '''optimizer = torch.optim.Adam([
      dict(params=model.reg_params, weight_decay=5e-4),
      dict(params=model.non_reg_params, weight_decay=0)
  ], lr=0.01)'''
  #cora_final_gcn
  optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)
  #pubmed_gcn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #cite_seer_gcn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gcns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7430
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7550
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7700
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7630
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7320
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7750
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7520
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7440
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7590
Net(
  (conv1): GCNConv(193, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.7640


In [41]:
gcns

[0.743, 0.755, 0.77, 0.763, 0.732, 0.775, 0.752, 0.744, 0.759, 0.764]

In [42]:
np.mean(gcns)

0.7557

## SAGE

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(num_features, 64, normalize=False)
        self.conv2 = SAGEConv(64, num_classes, normalize=False)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [42]:
sages = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  '''optimizer = torch.optim.Adam([
      dict(params=model.reg_params, weight_decay=5e-4),
      dict(params=model.non_reg_params, weight_decay=0)
  ], lr=0.01)'''
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-2)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)
  #pubmed
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #cite_seer_sage
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  sages.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 3)
)

Test Accuracy 0.7720
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 3)
)


KeyboardInterrupt: ignored

In [40]:
sages

[0.785, 0.789, 0.785, 0.786, 0.785, 0.785, 0.775, 0.79, 0.777, 0.793]

In [41]:
np.mean(sages)

0.785

## AGNN 

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = torch.nn.Linear(num_features, 64)
        self.prop1 = AGNNConv(requires_grad=True)
        self.prop2 = AGNNConv(requires_grad=True)
        self.lin2 = torch.nn.Linear(64, num_classes)

    def forward(self):
        x = F.dropout(data.x, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.prop1(x, data.edge_index)
        x = self.prop2(x, data.edge_index)
        x = F.dropout(x, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=1)

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [53]:
agnns = []
for _ in range(1):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-6)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #pubmed
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-3)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-4)
  #cite_seer
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-2)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print()
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  agnns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (lin1): Linear(in_features=193, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=7, bias=True)
)

Test Accuracy 0.7800


In [49]:
agnns

[0.772, 0.792, 0.794, 0.773, 0.786, 0.79, 0.783, 0.794, 0.776, 0.795]

In [50]:
np.mean(agnns)

0.7855000000000001

## GAT

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(num_features, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(
            8 * 8, num_classes, heads=1, concat=True, dropout=0.6)

    def forward(self):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(data.x, data.edge_index)) 
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index) 
        return F.log_softmax(x, dim=1)

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(num_features, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(
            8*8, num_classes, heads=8, concat=True, dropout=0.6)

    def forward(self):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(data.x, data.edge_index)) 
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index) 
        return F.log_softmax(x, dim=1)

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [55]:
gats = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora_gat
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)
  
  #pubmed_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=0.001)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-4)

  #cite_seer_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)
  #不行optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gats.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7750
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7680
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7820
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7660
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7780
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7640
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7830
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7810
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7750
Net(
  (conv1): GATConv(193, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)

Test Accuracy 0.7750


In [56]:
gats

[0.775, 0.768, 0.782, 0.766, 0.778, 0.764, 0.783, 0.781, 0.775, 0.775]

In [57]:
np.mean(gats)

0.7747

## GIN

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
hidden_dim = 64

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #nn1 = Sequential(Linear(num_features, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim))
        nn1 = Sequential(Linear(num_features, hidden_dim), ReLU())
        self.conv1 = GINConv(nn1)

        #nn2 = Sequential(Linear(hidden_dim, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim))
        nn2 = Sequential(Linear(hidden_dim, hidden_dim), ReLU())
        self.conv2 = GINConv(nn2)
        
        #nn3 = Sequential(Linear(hidden_dim, hidden_dim), ReLU())
        #self.conv3 = GINConv(nn3)
        
        self.fc1 = Linear(hidden_dim, hidden_dim)
        self.fc2 = Linear(hidden_dim, num_classes)

    def forward(self):
        x = data.x
        edge_index = data.edge_index
        x = F.relu(self.conv1(x, edge_index))
    
        x = F.relu(self.conv2(x, edge_index))
        
        #x = F.relu(self.conv3(x, edge_index))
      
        #x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [0]:
from torch.nn import Sequential, Linear, ReLU

In [0]:
gins = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #cora_gin
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-2)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
  
  #pubmed_gin
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-3)
  

  #cite_seer_gin
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)
  

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gins.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=500, out_features=64, bias=True)
    (1): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  ))
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.7590
Net(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=500, out_features=64, bias=True)
    (1): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  ))
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.7590
Net(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=500, out_features=64, bias=True)
    (1): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  ))
  (fc1

In [0]:
gins

[0.759, 0.759, 0.765, 0.746, 0.754, 0.786, 0.736, 0.762, 0.742, 0.772]

## Chebyshev too slow！

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = ChebConv(data.num_features, 64, K=2)
        self.conv2 = ChebConv(64, data.num_features, K=2)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

In [0]:
data = cora
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
chebs = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  '''optimizer = torch.optim.Adam([
      dict(params=model.reg_params, weight_decay=5e-4),
      dict(params=model.non_reg_params, weight_decay=0)
  ], lr=0.01)'''
  #cora_cheb
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #pubmed_
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #cite_seer_
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 21):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  chebs.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.8010
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7950
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7830
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7840
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7690
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7860
Net(
  (conv1): ChebConv(1433, 64, K=2, normalization=sym)
  (conv2): ChebConv(64, 1433, K=2, normalization=sym)
)

Test Accuracy 0.7730
Net(
  (conv1): ChebConv(1433, 64, K=2, n

KeyboardInterrupt: ignored